In [5]:
import google.cloud.aiplatform as aiplatform

In [6]:

import google.generativeai as genai

# Directly set the API key instead of using 'userdata'
gemini_key = ""  # Replace this with your actual API key

# Configure the library with the API key
genai.configure(api_key=gemini_key)


In [7]:
generation_config = {
  "temperature": 0.5,
  "top_p": 0.9,
  "top_k": 3,
  "max_output_tokens": 1500,
}


In [8]:
import pandas as pd
df=pd.read_csv("/content/cti-vsp.tsv",sep="\t")

In [9]:
df.head(10)

,URL,Description,Prompt,GT
0,https://nvd.nist.gov/vuln/detail/CVE-2024-23848,"In the Linux kernel through 6.7.1, there is a ...",Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:N/I:N/A:H
1,https://nvd.nist.gov/vuln/detail/CVE-2023-38738,IBM OpenPages with Watson 8.3 and 9.0 could pr...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:N
2,https://nvd.nist.gov/vuln/detail/CVE-2024-22137,Improper Neutralization of Input During Web Pa...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N
3,https://nvd.nist.gov/vuln/detail/CVE-2024-20819,Out-of-bounds Write vulnerabilities in svc1td_...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
4,https://nvd.nist.gov/vuln/detail/CVE-2024-0585,The Essential Addons for Elementor – Best Elem...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N
5,https://nvd.nist.gov/vuln/detail/CVE-2022-4958,A vulnerability classified as problematic has ...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N
6,https://nvd.nist.gov/vuln/detail/CVE-2023-41776,There is a local privilege escalation vulnerab...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H
7,https://nvd.nist.gov/vuln/detail/CVE-2022-40700,Server-Side Request Forgery (SSRF) vulnerabili...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
8,https://nvd.nist.gov/vuln/detail/CVE-2024-24570,Statamic is a Laravel and Git powered CMS. HTM...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N
9,https://nvd.nist.gov/vuln/detail/CVE-2024-0690,An information disclosure flaw was found in an...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:N/A:N


In [10]:
model = genai.GenerativeModel(model_name= "gemini-1.0-pro",
                              generation_config = generation_config)

In [11]:
import time

def get_cvss_score(description, model):
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. Analyze the following CVE "
        "description and calculate the CVSS v3.1 Base Score. Determine the values for each base metric: AV, AC, PR, UI, "
        "S, C, I, and A. Summarize each metric’s value and provide the final CVSS v3.1 vector string in the format:\n"
        "CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H\n\n"
        f"CVEDescription: {description}\nCVSS Vector:"
    )

    max_retries = 5
    backoff_time = 10  # initial backoff time in seconds

    for attempt in range(max_retries):
        try:
            # Generate the response
            response = model.generate_content(prompt)  # Replace with actual API call

            # Log the full response for debugging
            print(f"Response object: {response}")

            # Ensure response has 'parts' and is not empty
            if not hasattr(response, 'parts') or not response.parts:
                raise ValueError("No valid response parts found.")

            # Extract the text from the response object
            text = response.parts[0].text
            answer = text.strip().split()[-1].upper()  # Extract the last word and convert to uppercase
            return answer

        except TooManyRequests:  # Assuming TooManyRequests is a valid exception
            if attempt < max_retries - 1:
                sleep_time = backoff_time * (2 ** attempt)  # Exponential backoff
                print(f"Rate limit hit. Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                raise
        except Exception as e:
            print(f"An error occurred: {e}")
            raise


In [12]:
# Assuming 'model' is already defined and available
# Initialize the output data list

df1= df.head(200)


output_data = []
# Iterate over the sampled DataFrame rows
for index, row in df.iterrows():
    Description = row['Description']

    # Get the model response for the current Description
    try:
        response_text = get_cvss_score(Description, model)
        print(f"Description {index}: {response_text}")

        # Append the Description and response to the output_data list
        output_data.append({
            'Description': Description,
            'Geminai': response_text
        })
    except Exception as e:
        print(f"Failed to get a response for Description {index}: {e}")
        continue

# Create a new DataFrame from the output_data
output_df = pd.DataFrame(output_data)

# Save the new DataFrame to a CSV file
output_df.to_csv('geminaiCVSSresp.tsv',sep='\t', index=False)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "SAFETY",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "MEDIUM"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 274,
        "total_token_count": 274
      }
    }),
)
Failed to get a response for Description 867: name 'TooManyRequests' is not defined
Response object: res

In [23]:
import pandas as pd

# Load the predictions file and ground truth file, with bad lines skipped
predictions_path = '/content/geminaiCVSSresp.tsv'  # Replace with actual path to your predictions file
ground_truth_path = '/content/cti-vsp.tsv'  # Replace with actual path to the ground truth file

# Read the TSV files and skip lines with issues
gt_df = pd.read_csv(ground_truth_path, sep='\t', on_bad_lines='skip')
gemini_df = pd.read_csv(predictions_path, sep='\t', on_bad_lines='skip')

# Clean and standardize the predictions by removing asterisks if present
gemini_df['Geminai'] = gemini_df['Geminai'].str.replace(r'[*]', '', regex=True).str.strip()

# Merge dataframes on the 'Description' column
merged_df = pd.merge(gt_df[['Description', 'GT']], gemini_df[['Description', 'Geminai']], on='Description', how='inner')

# Calculate accuracy
accuracy = (merged_df['GT'] == merged_df['Geminai']).mean() * 100
print(f"Accuracy: {accuracy:.2f}%")



Accuracy: 50.34%
